Import necessary libraries

In [4]:
!pip install numpy pandas matplotlib scikit-learn jupyter


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

pd.options.display.float_format = '{:20.2f}'.format
pd.set_option('display.max_columns', 999)

# Data exploration

In [9]:
df = pd.read_excel("online_retail_II.xlsx", sheet_name=0)

df.head(10)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

## Explore the data using pandas functions such as `head()`, `info()`, `describe()`, `value_counts()`, etc.

In [ ]:
# include = 'all' will show all columns
# include = 'O' will show only object columns

In [ ]:
# the number of customer Ids are less than the number of rows -> there are missing values in the Customer ID column

In [ ]:
# Checking if invoice is a string of 6 digits

In [ ]:
# What other types are there, apart from 6 digits

In [ ]:
# A = Adjust bad debt 

In [ ]:
# checking stock codes
# a lot of stock codes which don't fall under the pattern of 5 digits

## Stock Codes note
- 5 digits = normal pattern
- POST = POSTAGE fees - ignore
- DOT = DOTCOM POSTAGE fees - ignore
- D = Discount? - ignore 
- DCGS = ??? - all customer IDs are null - ignore
- M = on its own, manual transaction - exclude
- M (at end) = looks fine - ignore
- BANK - bank charges - ignore (all quantity negative)
- TEST - test product - ignore
- git - giftshop voucher (Dotcomgiftshop) - ignore
- PADS - cushion pads (free price to match cushions but has customer ID) - allow
- ADJUST - adjustments made  but some have negative quantity - ignore
- S = samples - ignore
- B = adjust bad debt - ignore
- AMZ - Amazon fees probably for shipping - ignore


# Data Cleaning

In [ ]:
cleaned_df = df.copy()

In [ ]:
# clean invoice

In [ ]:
# clean stock code

In [ ]:
# clean customer ID

In [ ]:
# remove 0 price as there are only total of 28 rows with this

In [ ]:
cleaned_df.describe()

In [ ]:
# dropped around 23% of records
len(cleaned_df) / len(df)

# Feature Engineering


In [ ]:
# get sales line total from quantity and price

cleaned_df

In [ ]:
# create aggregated df grouped by each customer


In [ ]:
aggregated_df.head()

In [ ]:
# get max invoice date
max_invoice_date = aggregated_df["LastInvoiceDate"].max()

max_invoice_date # for purposes of project, make recency calculation from this date

In [ ]:
aggregated_df["Recency"] = (max_invoice_date - aggregated_df["LastInvoiceDate"]).dt.days

In [ ]:
aggregated_df.head()

In [ ]:
# histogram plot of different features

plt.figure(figsize=(15,5))

plt.subplot(1, 3, 1)
sns.histplot(aggregated_df["MonetaryValue"], bins=10, color="skyblue", edgecolor="black")
plt.title("Monetary Value Distribution")
plt.xlabel("Monetary Value")
plt.ylabel("Count")

plt.subplot(1, 3, 2)
sns.histplot(aggregated_df["Frequency"], bins=10, color="olive", edgecolor="black")
plt.title("Frequency Distribution")
plt.xlabel("Frequency")
plt.ylabel("Count")

plt.subplot(1, 3, 3)
sns.histplot(aggregated_df["Recency"], bins=10, color="gold", edgecolor="black")
plt.title("Recency Distribution")
plt.xlabel("Recency")
plt.ylabel("Count")

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1, 3, 1)
sns.boxplot(aggregated_df["MonetaryValue"], color="skyblue")
plt.title("Monetary Value BoxPlot")
plt.xlabel("Monetary Value")

plt.subplot(1, 3, 2)
sns.boxplot(aggregated_df["Frequency"], color="olive")
plt.title("Frequency BoxPlot")
plt.xlabel("Frequency")

plt.subplot(1, 3, 3)
sns.boxplot(aggregated_df["Recency"], color="gold")
plt.title("Recency BoxPlot")
plt.xlabel("Recency")

plt.tight_layout()
plt.show()

don't want to reject outliers in monetary and frequency as they are possibly one of the most important customers

In [ ]:
# separate out the outlier range
M_Q1 = aggregated_df["MonetaryValue"].quantile(0.25)
M_Q3 = aggregated_df["MonetaryValue"].quantile(0.75)
M_IQR = M_Q3 - M_Q1
M_outlier_range = (M_Q1 - 1.5 * M_IQR, M_Q3 + 1.5 * M_IQR)

F_Q1 = aggregated_df["Frequency"].quantile(0.25)
F_Q3 = aggregated_df["Frequency"].quantile(0.75)
F_IQR = F_Q3 - F_Q1
F_outlier_range = (F_Q1 - 1.5 * F_IQR, F_Q3 + 1.5 * F_IQR)

In [ ]:
monetary_outliers = aggregated_df[(aggregated_df["MonetaryValue"] < M_outlier_range[0]) | (aggregated_df["MonetaryValue"] > M_outlier_range[1])]
frequency_outliers = aggregated_df[(aggregated_df["Frequency"] < F_outlier_range[0]) | (aggregated_df["Frequency"] > F_outlier_range[1])]

In [ ]:
monetary_outliers.describe()

In [ ]:
frequency_outliers.describe()

In [ ]:

non_outliers = aggregated_df[(~aggregated_df.index.isin(monetary_outliers.index)) 
                             & (~aggregated_df.index.isin(frequency_outliers.index))] 

In [ ]:
non_outliers.describe()

In [ ]:
fig = plt.figure(figsize=(8,8))

ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(non_outliers["Recency"], non_outliers["Frequency"], non_outliers["MonetaryValue"], c=non_outliers["MonetaryValue"])

ax.set_xlabel("Recency")
ax.set_ylabel("Frequency")
ax.set_zlabel("Monetary Value")

plt.title("3D Scatter Plot of Recency, Frequency and Monetary Value")
plt.show()

In [ ]:
# rescaling to prevent monetary value being over emphasized
# using std deviation to scale the values - assumes normal distribution (which is not case but will do for now) - not very skewed data



In [ ]:
# plot scaled data

fig = plt.figure(figsize=(8,8))

ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(scaled_data_df["Recency"], scaled_data_df["Frequency"], scaled_data_df["MonetaryValue"], c=scaled_data_df["MonetaryValue"])

ax.set_xlabel("Recency")
ax.set_ylabel("Frequency")
ax.set_zlabel("Monetary Value")

plt.title("3D Scatter Plot of Scaled Recency, Frequency and Monetary Value")
plt.show()

# K Means Clustering

In [ ]:
# elbow method to determine optimal number of clusters
import sklearn


plt.figure(figsize=(14,6))

plt.subplot(1, 2, 1)
plt.plot(k_values, inertia, 'bo-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Elbow Method showing the optimal k')

plt.subplot(1, 2, 2)
plt.plot(k_values, silhouette_scores, 'bo-', color="orange")
plt.xlabel('k')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score showing the optimal k')
plt.show()

elbow of inertia curve around 4-5 - maximum benefit between 4 or 5 clusters
silhouette score is better at 4 than 5 = less overlap of clusters 
Thus using m=4

silhouette score to determine optimal number of clusters

s(i) = b(i) - a(i) / max(a(i), b(i))

- a(i) = average distance between i and all other points in the same cluster
- b(i) = smallest distance between i and all other points in the other clusters

silhouette score ranges from -1 to 1
- -1 means absolutely no overlapping between clusters
- 1 means perfect clustering (all overlapping)

In [ ]:


cluster_labels

In [ ]:
non_outliers["Cluster"] = cluster_labels

non_outliers.head()

In [ ]:
# visualising the clusters

cluster_colors = {0: "red", 1: "blue", 2: "green", 3: "purple"}
colors = non_outliers["Cluster"].map(cluster_colors)

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(non_outliers["Recency"], 
                     non_outliers["Frequency"], 
                     non_outliers["MonetaryValue"], 
                     c=colors,
                     marker='o')

ax.set_xlabel("Recency")
ax.set_ylabel("Frequency")
ax.set_zlabel("Monetary Value")

plt.title("3D Scatter Plot of Scaled Recency, Frequency and Monetary Value with Clusters")
plt.show()

In [ ]:
# violin plots of clusters

plt.figure(figsize=(12,18))

plt.subplot(3, 1, 1)
sns.violinplot(x="Cluster", y="MonetaryValue", data=non_outliers, palette=cluster_colors, hue =non_outliers["Cluster"])
sns.violinplot(y=non_outliers["MonetaryValue"], color="gray", linewidth=1.0)
plt.title("Monetary Value Distribution by Cluster")
plt.ylabel("Monetary Value")

plt.subplot(3, 1, 2)
sns.violinplot(x="Cluster", y="Frequency", data=non_outliers, palette=cluster_colors, hue =non_outliers["Cluster"])
sns.violinplot(y=non_outliers["Frequency"], color="gray", linewidth=1.0)
plt.title("Frequency Distribution by Cluster")
plt.ylabel("Frequency")

plt.subplot(3, 1, 3)
sns.violinplot(x="Cluster", y="Recency", data=non_outliers, palette=cluster_colors, hue =non_outliers["Cluster"])
sns.violinplot(y=non_outliers["Recency"], color="gray", linewidth=1.0)
plt.title("Recency Distribution by Cluster")
plt.ylabel("Recency")

plt.tight_layout()
plt.show()

# Interpret Clusters and give them meaningful names

1. Cluster 0 (Red): "Retain"
- Rationale: high-value customers who purchase regularly, though not always very recently. Focus on retention efforts to maintain loyalty and spending levels
- Action: implement loyalty programs, personalised offers, and regular engagement to ensure they remain active

2. Cluster 1 (Blue): "Re-Engage"
- Rationale: lower-value, infrequenet buyers who haven't purchased recently. Focus on re-engagement to bring them back into active purchasing behaviour
- Action@ targeted marketing campaigns, special discounts, reminders to engcourage

3. Cluster 2 (Green): "Nurture"
- Rationale: least active, lowest value customers but have made recent purchases. May be new or need nurturing to increase engagement
- Action: customer service, offer incentives to encourage more frequent purchases

4. Cluster 3 (Yellow): "Reward"
- Rationale: high value, very frequent buyers, many who are still actively purchasing. Most loyal customers and should reward to keep engagement
- Action: robust loyalty program, exclusive offers, early access to new products

# Conclusion
- Cluster 0 (Red): "Retain"
- Cluster 1 (Blue): "Re-Engage"
- Cluster 2 (Green): "Nurture"
- Cluster 3 (Yellow): "Reward"

# Outlier Analysis

convert into monetary, frequency, and both

In [ ]:
overlap_indices = monetary_outliers.index.intersection(frequency_outliers.index)

overlap_indices

In [ ]:
monetary_only_outliers = monetary_outliers.drop(overlap_indices)
frequency_only_outliers = frequency_outliers.drop(overlap_indices)
monetary_and_frequency_outliers = monetary_outliers.loc[overlap_indices]

monetary_only_outliers["Cluster"] = -1
frequency_only_outliers["Cluster"] = -2
monetary_and_frequency_outliers["Cluster"] = -3

In [ ]:
outlier_clusters = pd.concat([monetary_only_outliers, frequency_only_outliers, monetary_and_frequency_outliers])

In [ ]:
# visualising the outlier clusters using violinplot

cluster_colors = {-1: "red", -2: "blue", -3: "green"}
colors = outlier_clusters["Cluster"].map(cluster_colors)

plt.figure(figsize=(12,18))

plt.subplot(3, 1, 1)
sns.violinplot(x="Cluster", y="MonetaryValue", data=outlier_clusters, palette=cluster_colors, hue =outlier_clusters["Cluster"])
sns.violinplot(y=outlier_clusters["MonetaryValue"], color="gray", linewidth=1.0)
plt.title("Monetary Value Distribution by Cluster")
plt.ylabel("Monetary Value")

plt.subplot(3, 1, 2)
sns.violinplot(x="Cluster", y="Frequency", data=outlier_clusters, palette=cluster_colors, hue =outlier_clusters["Cluster"])
sns.violinplot(y=outlier_clusters["Frequency"], color="gray", linewidth=1.0)
plt.title("Frequency Distribution by Cluster")
plt.ylabel("Frequency")

plt.subplot(3, 1, 3)
sns.violinplot(x="Cluster", y="Recency", data=outlier_clusters, palette=cluster_colors, hue =outlier_clusters["Cluster"])
sns.violinplot(y=outlier_clusters["Recency"], color="gray", linewidth=1.0)
plt.title("Recency Distribution by Cluster")
plt.ylabel("Recency")

plt.tight_layout()
plt.show()


# Analysis for Outliers

- Cluster -1 (Monetary Outliers) PAMPER : High money spenders but low frequency. Recency ranges a lot but most are fairly recent. Possibly one time buyers of bulky orders. Either loyalty programs or luxury services

- Cluster -2 (Frequency Outliers) UPSELL : High frequency buyers who do not spend much. Majority of the customers buy the products fairly recently but there are some who have not bought in a long time. They may be buying low value items or may be buying in bulk. Targeted marketing campaigns or discounts may help increase their spending. Possibly bundle deals or discounts on bulk orders.

- Cluster -3 (High Value Outliers) DELIGHT : customers who spend a lot and buy very frequently. Although most have stayed on recently, there are some who have not bought in a long time. They are the most valuable customers and should be rewarded to keep them engaged. VIP programs or exclusive offers.

In [ ]:
cluster_lables = {
    0: "RETAIN",
    1: "RE-ENGAGE",
    2: "NURTURE",
    3: "REWARD",
    -1: "PAMPER",
    -2: "UPSELL",
    -3: "DELIGHT"
}

In [ ]:
full_clustering = pd.concat([non_outliers, outlier_clusters])

In [ ]:
full_clustering["ClusterLabel"] = full_clustering["Cluster"].map(cluster_lables)

# Visualisation

In [ ]:
cluster_counts = full_clustering["ClusterLabel"].value_counts()
full_clustering["MonetaryValue per 100 pounds"] = full_clustering["MonetaryValue"] / 100
feature_means = full_clustering.groupby("ClusterLabel")[["Recency", "Frequency", "MonetaryValue per 100 pounds"]].mean()

fig1, ax1 = plt.subplots(figsize=(12, 8))

sns.barplot(x=cluster_counts.index, y=cluster_counts.values, ax=ax1, palette="viridis", hue=cluster_counts.index)
ax1.set_ylabel("Number of Customers", color="blue")
ax1.set_title("Cluster Distribution with Average Feature Values")

ax2 = ax1.twinx()

sns.lineplot(data=feature_means, ax=ax2, palette="Set2", marker='o')
ax2.set_ylabel("Average Value", color="red")

plt.show()